In [1]:
import xarray as xr
import pandas as pd

print('Pandas version', pd.__version__)
print('Xarray version', xr.__version__)

Pandas version 1.1.3
Xarray version 0.16.1


# xarray.Dataset
- MCDS is created as a xarray.Dataset;
- MCDS is saved in netCDF4 format using xarray.Dataset.to_netcdf();
- netCDF4 is a special kind of HDF5 format, you do not need to know the details of it before using MCDS.
- To store raw methylation counts, MCDS usually contain multiple dimensions:
    - cell: individual cell, coordinates are cell ids.
    - count_type: type of the count, coordinates are fixed:
        mc: the methyl-cytosine base count
        cov(erage): the total cytosine base count
    - mc_type: type of the methylation context, coordinates are usually CHN(mCH) or CGN(mCG).
    - strand_type: type of the strand information, usually only has "both", means all counts summed from both strands.
    - chrom?k: individual non-overlapping genomic bin, we usually use 100-kb (chrom100k) as features for clustering.
    - gene: individual gene


## Open MCDS

### Single MCDS

In [15]:
mcds = xr.open_dataset('./Example.mcds')
mcds

<xarray.Dataset>
Dimensions:              (cell: 20, chrom100k: 10000, count_type: 2, gene: 10000, mc_type: 2, strand_type: 1)
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG000...
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object ...
    chrom100k_bin_start  (chrom100k) int64 ...
    chrom100k_bin_end    (chrom100k) int64 ...
    gene_chrom           (gene) object ...
    gene_start           (gene) int64 ...
    gene_end             (gene) int64 ...
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 ...
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 ...

### Multiplex MCDS

- MCDS usually contain cells from a single experiment/batch;
- The clustering analysis usually needs to start from all available cells, stored in multiple MCDS;
- xarray allows open multiplex MCDS in one command and automatically concatenate them by the "cell" dimension;
- This do requires MCDS having the same feature structure.

In [42]:
mcds = xr.open_mfdataset(['./Example.mcds'], # provide a list of mcds paths here, and they will be autom
                         concat_dim='cell')
mcds

<xarray.Dataset>
Dimensions:              (cell: 20, chrom100k: 10000, count_type: 2, gene: 10000, mc_type: 2, strand_type: 1)
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG000...
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object dask.array<open_dataset-8b63df073...
    chrom100k_bin_start  (chrom100k) int64 dask.array<open_dataset-8b63df0730...
    chrom100k_bin_end    (chrom100k) int64 dask.array<open_dataset-8b63df0730...
    gene_chrom           (gene) object dask.array<open_dataset-8b63df07305a2b...
    gene_start           (gene) int64 dask.array<open_dataset-8b63df07305a2ba...
    gene_end             (gene) int64 dask.array<open_dataset-8b63df07305a2ba...
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 dask.array<open_dataset-8b63df07305a2ba4e0241826437220d4gene_da, shape=(20, 10000, 2, 1, 2), dtype=uint16, chunksize=(20, 10000, 2, 1, 2), chunktype=numpy.ndarra...
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 dask.array<open_dataset-8b63df07305a2ba4e0241826437220d4chrom100k_da, shape=(20, 10000, 2, 1, 2), dtype=uint16, chunksize=(20, 10000, 2, 1, 2), chunktype=numpy.n...

## Index

### Get index

In [18]:
cells = mcds.get_index('cell')
cells

Index(['3C_M_1015', '3C_M_0', '3C_M_1005', '3C_M_1', '3C_M_1004', '3C_M_1002',
       '3C_M_1016', '3C_M_1017', '3C_M_1012', '3C_M_1013', '3C_M_1018',
       '3C_M_1003', '3C_M_1001', '3C_M_1011', '3C_M_1000', '3C_M_1009',
       '3C_M_1014', '3C_M_1006', '3C_M_1007', '3C_M_100'],
      dtype='object', name='cell')

In [19]:
genes = mcds.get_index('gene')
genes

Index(['ENSMUSG00000102693.1', 'ENSMUSG00000064842.1', 'ENSMUSG00000051951.5',
       'ENSMUSG00000102851.1', 'ENSMUSG00000103377.1', 'ENSMUSG00000104017.1',
       'ENSMUSG00000103025.1', 'ENSMUSG00000089699.1', 'ENSMUSG00000103201.1',
       'ENSMUSG00000103147.1',
       ...
       'ENSMUSG00000105357.1', 'ENSMUSG00000070342.3', 'ENSMUSG00000106515.4',
       'ENSMUSG00000105797.1', 'ENSMUSG00000089269.1', 'ENSMUSG00000104785.1',
       'ENSMUSG00000047676.6', 'ENSMUSG00000105779.1', 'ENSMUSG00000105190.1',
       'ENSMUSG00000092830.1'],
      dtype='object', name='gene', length=10000)

### Select subset using index

In [37]:
# only select 10 cells
mcds.sel(cell=cells[:10])

<xarray.Dataset>
Dimensions:              (cell: 10, chrom100k: 10000, count_type: 2, gene: 10000, mc_type: 2, strand_type: 1)
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_1013'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG000...
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000
    gene_chrom           (gene) object 'chr1' 'chr1' 'chr1' ... 'chr3' 'chr3'
    gene_start           (gene) int64 3073252 3102015 ... 150588738 151023791
    gene_end             (gene) int64 3074321 3102124 ... 150612175 151023861
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 ...
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 ...

In [38]:
# only select 100 genes
mcds.sel(gene=genes[:100])

<xarray.Dataset>
Dimensions:              (cell: 20, chrom100k: 10000, count_type: 2, gene: 100, mc_type: 2, strand_type: 1)
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG000...
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000
    gene_chrom           (gene) object 'chr1' 'chr1' 'chr1' ... 'chr1' 'chr1'
    gene_start           (gene) int64 3073252 3102015 ... 9450578 9458669
    gene_end             (gene) int64 3074321 3102124 ... 9451747 9458771
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 ...
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 ...

In [39]:
# apply multiple selection together
mcds.sel(cell=cells[:10], gene=genes[:100])

<xarray.Dataset>
Dimensions:              (cell: 10, chrom100k: 10000, count_type: 2, gene: 100, mc_type: 2, strand_type: 1)
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_1013'
  * gene                 (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG000...
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000
    gene_chrom           (gene) object 'chr1' 'chr1' 'chr1' ... 'chr1' 'chr1'
    gene_start           (gene) int64 3073252 3102015 ... 9450578 9458669
    gene_end             (gene) int64 3074321 3102124 ... 9451747 9458771
Data variables:
    gene_da              (cell, gene, mc_type, strand_type, count_type) uint16 ...
    chrom100k_da         (cell, chrom100k, mc_type, strand_type, count_type) uint16 ...

## Select Data variable

### Select chrom100k data

In [22]:
chrom100k_da = mcds['chrom100k_da']
chrom100k_da

<xarray.DataArray 'chrom100k_da' (cell: 20, chrom100k: 10000, mc_type: 2, strand_type: 1, count_type: 2)>
[800000 values with dtype=uint16]
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
  * count_type           (count_type) object 'mc' 'cov'
  * strand_type          (strand_type) object 'both'
  * mc_type              (mc_type) object 'CGN' 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000

### Select gene data

In [23]:
gene_da = mcds['gene_da']
gene_da

<xarray.DataArray 'gene_da' (cell: 20, gene: 10000, mc_type: 2, strand_type: 1, count_type: 2)>
[800000 values with dtype=uint16]
Coordinates:
  * cell         (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_1007' '3C_M_100'
  * gene         (gene) object 'ENSMUSG00000102693.1' ... 'ENSMUSG00000092830.1'
  * count_type   (count_type) object 'mc' 'cov'
  * strand_type  (strand_type) object 'both'
  * mc_type      (mc_type) object 'CGN' 'CHN'
    gene_chrom   (gene) object 'chr1' 'chr1' 'chr1' ... 'chr3' 'chr3' 'chr3'
    gene_start   (gene) int64 3073252 3102015 3205900 ... 150588738 151023791
    gene_end     (gene) int64 3074321 3102124 3671497 ... 150612175 151023861

## Select 2-D DataArray of different kinds

In [26]:
# mc count of mCH
ch_mc = chrom100k_da.sel(mc_type='CHN', count_type='mc').squeeze()
ch_mc

<xarray.DataArray 'chrom100k_da' (cell: 20, chrom100k: 10000)>
[200000 values with dtype=uint16]
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
    count_type           <U2 'mc'
    strand_type          <U4 'both'
    mc_type              <U3 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000

In [33]:
# 2-D DataArray to pandas DataFrame
ch_mc_df = ch_mc.to_pandas()
ch_mc_df

chrom100k,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
cell,,,,,,,,,,,,,,,,,,,,,
3C_M_1015,0,0,0,0,0,0,0,0,0,0,...,0,1,14,9,0,0,0,0,0,0
3C_M_0,0,0,0,0,0,0,0,0,0,0,...,0,4,6,13,4,26,15,23,4,6
3C_M_1005,0,0,0,0,0,0,0,0,0,0,...,0,3,3,18,42,63,21,20,6,2
3C_M_1,0,0,0,0,0,0,0,0,0,0,...,0,8,25,36,19,44,21,22,14,6
3C_M_1004,0,0,0,0,0,0,0,0,0,0,...,0,6,15,25,14,48,17,26,6,3
3C_M_1002,0,0,0,0,0,0,0,0,0,0,...,0,8,18,42,38,38,18,27,4,2
3C_M_1016,0,0,0,0,0,0,0,0,0,0,...,0,9,14,27,11,45,19,22,10,3
3C_M_1017,0,0,0,0,0,0,0,0,0,0,...,0,1,14,59,17,64,21,43,5,8
3C_M_1012,0,0,0,0,0,0,0,0,0,0,...,0,6,11,52,35,81,23,37,7,4


In [29]:
# similarly cov count of mCH
ch_cov = chrom100k_da.sel(mc_type='CHN', count_type='cov').squeeze()
ch_cov

<xarray.DataArray 'chrom100k_da' (cell: 20, chrom100k: 10000)>
[200000 values with dtype=uint16]
Coordinates:
  * cell                 (cell) object '3C_M_1015' '3C_M_0' ... '3C_M_100'
    count_type           <U3 'cov'
    strand_type          <U4 'both'
    mc_type              <U3 'CHN'
  * chrom100k            (chrom100k) int64 0 1 2 3 4 ... 9996 9997 9998 9999
    chrom100k_chrom      (chrom100k) object 'chr1' 'chr1' ... 'chr7' 'chr7'
    chrom100k_bin_start  (chrom100k) int64 0 100000 200000 ... 3700000 3800000
    chrom100k_bin_end    (chrom100k) int64 100000 200000 ... 3800000 3900000

In [32]:
# 2-D DataArray to pandas DataFrame
ch_cov_df = ch_cov.to_pandas()
ch_cov_df

chrom100k,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
cell,,,,,,,,,,,,,,,,,,,,,
3C_M_1015,0,0,0,0,0,0,0,0,0,0,...,0,446,1180,330,0,18,0,0,0,0
3C_M_0,0,0,0,0,0,0,0,0,0,0,...,0,280,567,472,194,725,433,2023,368,739
3C_M_1005,0,0,0,0,0,0,0,0,0,0,...,0,495,806,1004,1718,2170,926,1579,610,341
3C_M_1,0,0,0,0,0,0,0,0,0,0,...,0,761,1636,764,1183,1749,686,1904,540,375
3C_M_1004,0,0,0,0,0,0,0,0,0,0,...,0,958,2900,1744,2007,2437,946,1780,292,252
3C_M_1002,0,0,0,0,0,0,0,0,0,0,...,0,908,2301,2012,1874,2145,962,1789,683,645
3C_M_1016,0,0,0,0,0,0,0,0,0,0,...,0,865,1469,1336,1580,1702,600,2246,626,532
3C_M_1017,0,0,0,0,0,0,0,0,0,0,...,0,447,1875,2254,765,2269,858,2839,772,961
3C_M_1012,0,0,0,0,0,0,0,0,0,0,...,0,929,1630,1633,1548,2108,1031,2475,491,607


In [35]:
# you can save the pandas.DataFrame into any other table format, such as CSV
ch_cov_df.to_csv('Example.CHN.cov.csv.gz')